In [3]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

In [2]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, _ = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                metrics
            )
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels, 
                    names=['tuned_bm25_unjudged'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels, 
                    names=['tuned_bm25_unjudged20'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)].copy()], topics21, qrels, 
                    names=['tuned_bm25_unjudged21'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            

            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.580154
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.405992
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']


/tmp/ipykernel_2391427/3064225256.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels,
/tmp/ipykernel_2391427/3064225256.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels,


\begin{tabular}{llrrr}
\toprule
{} &                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged &    0.719094 &     0.711802 &  0.717687 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged20 &    0.777254 &     0.750278 &  0.703588 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391427/3064225256.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)].copy()], topics21, qrels,


\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged21 &    0.694795 &     0.721827 &  0.731503 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.607403
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.0', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.400238
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.0', 'BR(BM25) bm25.k_3=5']


...

/tmp/ipykernel_2391427/3064225256.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels,
/tmp/ipykernel_2391427/3064225256.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels,
/tmp/ipykernel_2391427/3064225256.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise t

\begin{tabular}{llrrr}
\toprule
{} &                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  tuned\_bm25\_unjudged &    0.730077 &     0.717043 &  0.71818 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  tuned\_bm25\_unjudged20 &    0.783887 &     0.754415 &  0.69792 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged21 &    0.722592 &     0.734256 &  0.738035 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusions_leakage/pd_index
Fold 1
Best ndcg_cut_5 is 0.607799
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.408486
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2

/tmp/ipykernel_2391427/3064225256.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels,
/tmp/ipykernel_2391427/3064225256.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels,
/tmp/ipykernel_2391427/3064225256.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise t

\begin{tabular}{llrrr}
\toprule
{} &                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged &    0.728717 &     0.715912 &  0.718364 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged20 &    0.785272 &     0.750293 &  0.704273 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged21 &    0.707772 &     0.726571 &  0.732173 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.593132
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.377257
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.0', 'BR(BM25) bm25.k_3=

/tmp/ipykernel_2391427/3064225256.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels,
/tmp/ipykernel_2391427/3064225256.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels,
/tmp/ipykernel_2391427/3064225256.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise t

In [4]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, _ = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                metrics
            )
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels, 
                    names=['tuned_bm25_unjudged'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels, 
                    names=['tuned_bm25_unjudged20'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)].copy()], topics21, qrels, 
                    names=['tuned_bm25_unjudged21'], 
                    eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            

            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.460400
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=3.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.278232
Best setting is ['BR(BM25) c=0.75', 'BR(BM25) bm25.k_1=0.6', 'BR(BM25) bm25.k_3=2']
\begin{tabular}{llrrr}
\toprule
{} &                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged &    0.556162 &     0.601818 &  0.622564 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged20 &    0.606965 &     0.637791 &  0.621228 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_bm25\_unjudged21 &    0.529812 &     0.609965 &  0.623873 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391427/3004830052.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)].copy()], topics, qrels,
/tmp/ipykernel_2391427/3004830052.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)].copy()], topics20, qrels,
/tmp/ipykernel_2391427/3004830052.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise t